In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,mls,mls
band,[8],6
commitnumber,1013f91,5014a19
conc,-,-
dv,-,0.001
klin,-,5e-25
molecule,{'h2o': 'atmpro'},h2o
ng_adju,-,[0]
ng_refs,-,[4]
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD               CLIRAD `coolr_bands`              
            layer        coolrg                layer        coolrg
pressure                                                          
0.000312        1 -2.325424e-05                    1 -1.510277e-06
0.000750        2 -2.305708e-05                    2  1.743909e-05
0.001052        3 -2.261218e-05                    3 -9.335788e-06
0.001476        4 -2.211744e-05                    4 -1.412500e-05
0.002070        5 -2.157677e-05                    5 -1.265928e-05
0.002904        6 -2.099863e-05                    6 -1.393818e-05
0.004074        7 -2.038466e-05                    7 -1.298307e-05
0.005714        8 -1.973184e-05                    8 -1.367870e-05
0.008015        9 -1.903195e-05                    9 -1.305498e-05
0.011243       10 -1.829873e-05                   10 -1.326649e-05
0.015771       11 -1.752827e-05                   11 -1.318763e-05
0.022122       12 -1.670660e-05                   12 -1.297835e-05
0.031031       13 -1.584098e-05                   13 -1.280534e-05
0.043528       14 -1.486255e-05                   14 -9.714715e-06
0.061057       15 -1.164532e-05                   15 -5.993378e-06
0.085645       16 -3.505994e-07                   16 -5.574007e-06
0.120136       17  2.437946e-05                   17 -2.759158e-06
0.168516       18  6.418822e-05                   18  6.040388e-06
0.236378       19  1.248647e-04                   19  2.037077e-05
0.331549       20  2.141946e-04                   20  4.204671e-05
0.465100       21  3.421664e-04                   21  7.380498e-05
0.652400       22  5.202400e-04                   22  1.191764e-04
0.915100       23  7.053925e-04                   23  1.665381e-04
1.283650       24  7.200040e-04                   24  1.720042e-04
1.800600       25  5.677113e-04                   25  1.349287e-04
2.525700       26  4.088647e-04                   26  9.495972e-05
3.542800       27  2.819393e-04                   27  6.266709e-05
4.969550       28  1.874858e-04                   28  3.849112e-05
6.970850       29  1.210936e-04                   29  2.128259e-05
9.778100       30  7.450739e-05                   30  8.968997e-06
13.715850      31  4.040506e-05                   31 -3.483871e-07
19.239350      32  1.602507e-05                   32 -7.340544e-06
26.987250      33  1.007268e-06                   33 -1.206344e-05
37.855300      34 -7.935108e-06                   34 -1.538327e-05
53.100050      35 -1.488345e-05                   35 -1.843448e-05
73.887500      36 -2.051687e-05                   36 -2.141625e-05
97.662500      37 -2.377461e-05                   37 -2.370145e-05
121.437500     38 -2.548166e-05                   38 -2.529784e-05
145.212500     39 -2.708272e-05                   39 -2.661717e-05
168.987500     40 -2.890967e-05                   40 -2.876588e-05
192.762500     41 -4.001848e-05                   41 -4.332601e-05
216.537500     42 -7.557149e-05                   42 -1.043217e-04
240.312500     43 -8.088149e-05                   43 -1.912307e-04
264.087500     44  1.834701e-05                   44 -2.621707e-04
287.862500     45  3.010791e-04                   45 -2.215017e-04
311.637500     46  7.257638e-04                   46  6.948308e-05
335.412500     47  1.296967e-03                   47  5.991813e-04
359.187500     48  2.078026e-03                   48  1.393072e-03
382.962500     49  3.100541e-03                   49  2.471651e-03
406.737500     50  4.391978e-03                   50  3.806775e-03
430.512500     51  5.969354e-03                   51  5.333313e-03
454.287500     52  7.709830e-03                   52  6.878030e-03
478.062500     53  9.690026e-03                   53  8.493209e-03
501.837500     54  1.200194e-02                   54  1.027436e-02
525.612500     55  1.467523e-02                   55  1.231597e-02
549.387500     56  1.774709e-02                   56  1.482824e-02
573.162500     57  2.128507e-02                   57  1.80026

# Fluxes by Layer

CRD                              CLIRAD  \
                        flug          fldg      fnetg       flug   
pressure    level                                                  
0.000000    1     -21.780996  0.000000e+00 -21.780996 -21.880781   
0.000624    2     -21.780996  1.378786e-09 -21.780996 -21.880781   
0.000876    3     -21.780996  1.942834e-09 -21.780996 -21.880781   
0.001229    4     -21.780996  2.760500e-09 -21.780996 -21.880781   
0.001723    5     -21.780996  3.948470e-09 -21.780996 -21.880781   
0.002417    6     -21.780996  5.673999e-09 -21.780996 -21.880781   
0.003391    7     -21.780996  8.180604e-09 -21.780996 -21.880781   
0.004757    8     -21.780996  1.182119e-08 -21.780996 -21.880781   
0.006672    9     -21.780996  1.710783e-08 -21.780996 -21.880781   
0.009359    10    -21.780996  2.478416e-08 -21.780996 -21.880781   
0.013128    11    -21.780996  3.592900e-08 -21.780996 -21.880781   
0.018415    12    -21.780996  5.210797e-08 -21.780996 -21.880781   
0.025830    13    -21.780996  7.559244e-08 -21.780996 -21.880781   
0.036232    14    -21.780996  1.096784e-07 -21.780996 -21.880781   
0.050823    15    -21.780996  1.592564e-07 -21.780996 -21.880781   
0.071291    16    -21.780996  2.365699e-07 -21.780996 -21.880781   
0.100000    17    -21.780996  3.798058e-07 -21.780996 -21.880781   
0.140271    18    -21.780997  6.741545e-07 -21.780996 -21.880781   
0.196760    19    -21.780997  1.276706e-06 -21.780996 -21.880781   
0.275997    20    -21.780997  2.496307e-06 -21.780994 -21.880781   
0.387100    21    -21.780997  4.935495e-06 -21.780992 -21.880781   
0.543100    22    -21.780995  9.761896e-06 -21.780985 -21.880781   
0.761700    23    -21.780991  1.915959e-05 -21.780972 -21.880780   
1.068500    24    -21.780982  3.606646e-05 -21.780946 -21.880779   
1.498800    25    -21.780969  6.004530e-05 -21.780909 -21.880777   
2.102400    26    -21.780956  8.737486e-05 -21.780869 -21.880775   
2.949000    27    -21.780944  1.164266e-04 -21.780828 -21.880775   
4.136600    28    -21.780935  1.466151e-04 -21.780788 -21.880775   
5.802500    29    -21.780929  1.777180e-04 -21.780751 -21.880778   
8.139200    30    -21.780927  2.100141e-04 -21.780718 -21.880783   
11.417000   31    -21.780932  2.439066e-04 -21.780689 -21.880792   
16.014700   32    -21.780947  2.794329e-04 -21.780667 -21.880807   
22.464000   33    -21.780972  3.168528e-04 -21.780654 -21.880828   
31.510500   34    -21.781012  3.584014e-04 -21.780654 -21.880859   
44.200100   35    -21.781074  4.077982e-04 -21.780665 -21.880905   
62.000000   36    -21.781164  4.676821e-04 -21.780697 -21.880974   
85.775000   37    -21.781293  5.380307e-04 -21.780755 -21.881071   
109.550000  38    -21.781426  6.049111e-04 -21.780822 -21.881174   
133.325000  39    -21.781566  6.725131e-04 -21.780893 -21.881284   
157.100000  40    -21.781711  7.408926e-04 -21.780970 -21.881398   
180.875000  41    -21.781867  8.158474e-04 -21.781051 -21.881524   
204.650000  42    -21.782117  9.538268e-04 -21.781164 -21.881728   
228.425000  43    -21.782811  1.433532e-03 -21.781377 -21.882308   
252.200000  44    -21.784412  2.806818e-03 -21.781604 -21.883687   
275.975000  45    -21.787632  6.079431e-03 -21.781552 -21.886526   
299.750000  46    -21.793450  1.274490e-02 -21.780705 -21.891736   
323.525000  47    -21.801633  2.297224e-02 -21.778660 -21.899135   
347.300000  48    -21.812160  3.715205e-02 -21.775008 -21.908692   
371.075000  49    -21.825364  5.620822e-02 -21.769155 -21.920733   
394.850000  50    -21.841569  8.114508e-02 -21.760423 -21.935578   
418.625000  51    -21.861077  1.130225e-01 -21.748055 -21.953510   
442.400000  52    -21.884187  1.529443e-01 -21.731243 -21.974796   
466.175000  53    -21.910643  2.011129e-01 -21.709530 -21.999218   
489.950000  54    -21.940376  2.581373e-01 -21.682239 -22.026716   
513.725000  55    -21.973537  3.250988e-01 -21.648438 -22.057458   
537.500000  56    -22.010238  4.031296e-01 -21.607108 -22.091520   
561.275000  5

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')